In [1]:
# 1 Inital Conf

In [2]:
import re
import json
from bs4 import BeautifulSoup
import requests
import pandas as pd


pd.options.display.max_columns = None

In [3]:
urlFundamental = 'https://finance.yahoo.com/quote/{0}/financials?p={1}'

In [4]:
stock = "ABEV3.SA"

In [5]:
# 2 Return HTML

In [6]:
url = urlFundamental.format(stock,stock)
url

'https://finance.yahoo.com/quote/ABEV3.SA/financials?p=ABEV3.SA'

In [7]:
headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.61 Safari/537.36'}
resp = requests.get(url,headers= headers)

In [8]:
brute = BeautifulSoup(resp.text, "html.parser")

In [9]:
# 3 Filter page

In [10]:
pattern = re.compile(r'\s--\sData\s--\s')
brute_datas = brute.find('script', text = pattern).contents[0]

In [11]:
brute_datas[:500]

'\n(function (root) {\n/* -- Data -- */\nroot.App || (root.App = {});\nroot.App.now = 1659816520140;\nroot.App.main = {"context":{"dispatcher":{"stores":{"PageStore":{"currentPageName":"quote","currentEvent":{"eventName":"NEW_PAGE_SUCCESS"},"currentRenderTargetId":"default","pagesConfigRaw":{"base":{"quote":{"layout":{"bundleName":"yahoodotcom-layout.TwoColumnLayout","name":"TwoColumnLayout","config":{"enableHeaderCollapse":true,"additionalBodyWrapperClasses":"Bgc($layoutBgColor)!","contentWrapperClas'

In [12]:
brute_datas[-500:]

'dv2-applet-sponsored-moments":{"strings":1},"tdv2-applet-stream":{"strings":1},"tdv2-applet-stream-hero":{"strings":1},"tdv2-applet-swisschamp":{"strings":1},"tdv2-applet-uh":{"strings":1},"tdv2-applet-userintent":{"strings":1},"tdv2-applet-video-lightbox":{"strings":1},"tdv2-applet-video-modal":{"strings":1},"tdv2-wafer-adfeedback":{"strings":1},"tdv2-wafer-header":{"strings":1},"xray":{"strings":1},"yahoodotcom-layout":{"strings":1}}},"options":{"defaultBundle":"td-app-finance"}}}};\n}(this));\n'

In [13]:
init = brute_datas.find("context")-2

In [14]:
dados_json =json.loads(brute_datas[init:-12])

In [15]:
type(dados_json)

dict

In [16]:
# 4 Explore struct

In [17]:
dados_json['context'].keys()

dict_keys(['dispatcher', 'options', 'plugins'])

In [18]:
dados_json['context']['dispatcher']['stores'].keys()

dict_keys(['PageStore', 'MRTStore', 'RouteStore', 'I13nStore', 'PageTransitionStore', 'VideoPlayerStore', 'QuoteAutoCompleteStore', 'NavrailStore', 'StreamDataStore', 'QuoteSummaryStore', 'FinanceConfigStore', 'LangStore', 'BeaconStore', 'AdStore', 'VideoStore', 'ComponentConfigStore', 'CrumbStore', 'CompositeStore', 'MonalixaStore', 'StreamStore', 'UserStore', 'ProfileStore', 'QuotePageStore', 'FinancialTemplateStore', 'QuoteTimeSeriesStore', 'MarketTimeStore', 'MarketSummaryStore', 'NavServiceStore', 'RmpStore', 'RecommendationStore', 'UHAccountSwitchStore', 'MobileHeaderStore'])

In [19]:
# amount stock
dados_json['context']['dispatcher']['stores']['QuoteTimeSeriesStore'].keys()

dict_keys(['timeSeries', 'meta', 'loading', 'errorList'])

In [72]:
dados_json['context']['dispatcher']['stores']['QuoteSummaryStore']["balanceSheetHistoryQuarterly"]

{'balanceSheetStatements': [{'intangibleAssets': {'raw': 8552850000,
    'fmt': '8.55B',
    'longFmt': '8,552,850,000'},
   'capitalSurplus': {'raw': 53662811000,
    'fmt': '53.66B',
    'longFmt': '53,662,811,000'},
   'totalLiab': {'raw': 49046382000,
    'fmt': '49.05B',
    'longFmt': '49,046,382,000'},
   'totalStockholderEquity': {'raw': 86263126000,
    'fmt': '86.26B',
    'longFmt': '86,263,126,000'},
   'minorityInterest': {'raw': 1323900000,
    'fmt': '1.32B',
    'longFmt': '1,323,900,000'},
   'otherCurrentLiab': {'raw': 6405327000,
    'fmt': '6.41B',
    'longFmt': '6,405,327,000'},
   'totalAssets': {'raw': 136633408000,
    'fmt': '136.63B',
    'longFmt': '136,633,408,000'},
   'endDate': {'raw': 1656547200, 'fmt': '2022-06-30'},
   'commonStock': {'raw': 58130517000,
    'fmt': '58.13B',
    'longFmt': '58,130,517,000'},
   'otherCurrentAssets': {'raw': 1980140000,
    'fmt': '1.98B',
    'longFmt': '1,980,140,000'},
   'retainedEarnings': {'raw': 39311455000,
   

In [21]:
dados_json['context']['dispatcher']['stores']['QuoteTimeSeriesStore']["timeSeries"]['annualBasicAverageShares'][0]["reportedValue"]["raw"]

15718100000

In [22]:
dados_json['context']['dispatcher']['stores']['QuoteSummaryStore'].keys()

dict_keys(['financialsTemplate', 'cashflowStatementHistory', 'balanceSheetHistoryQuarterly', 'earnings', 'price', 'incomeStatementHistoryQuarterly', 'incomeStatementHistory', 'balanceSheetHistory', 'cashflowStatementHistoryQuarterly', 'quoteType', 'summaryDetail', 'symbol', 'pageViews'])

In [23]:
# 5 Return Income Statement

In [24]:
dados_json['context']['dispatcher']['stores']['QuoteSummaryStore']['incomeStatementHistory'].keys()

dict_keys(['incomeStatementHistory', 'maxAge'])

In [25]:
income_statement = dados_json['context']['dispatcher']['stores']['QuoteSummaryStore']['incomeStatementHistory']['incomeStatementHistory']

In [26]:
print(income_statement[0])

{'researchDevelopment': {}, 'effectOfAccountingCharges': {}, 'incomeBeforeTax': {'raw': 13759160000, 'fmt': '13.76B', 'longFmt': '13,759,160,000'}, 'minorityInterest': {'raw': 1374586000, 'fmt': '1.37B', 'longFmt': '1,374,586,000'}, 'netIncome': {'raw': 12670968000, 'fmt': '12.67B', 'longFmt': '12,670,968,000'}, 'sellingGeneralAdministrative': {'raw': 21961211000, 'fmt': '21.96B', 'longFmt': '21,961,211,000'}, 'grossProfit': {'raw': 37194600000, 'fmt': '37.19B', 'longFmt': '37,194,600,000'}, 'ebit': {'raw': 17291355000, 'fmt': '17.29B', 'longFmt': '17,291,355,000'}, 'endDate': {'raw': 1640908800, 'fmt': '2021-12-31'}, 'operatingIncome': {'raw': 17291355000, 'fmt': '17.29B', 'longFmt': '17,291,355,000'}, 'otherOperatingExpenses': {'raw': -2057966000, 'fmt': '-2.06B', 'longFmt': '-2,057,966,000'}, 'interestExpense': {'raw': -1431774000, 'fmt': '-1.43B', 'longFmt': '-1,431,774,000'}, 'extraordinaryItems': {}, 'nonRecurring': {}, 'otherItems': {}, 'incomeTaxExpense': {'raw': 636578000, 'fm

In [27]:
financial_quarterly = []

In [28]:
for s in income_statement:
    balance = {}
    for key, val in s.items():
        try:
            balance[key] = val["raw"]
        except TypeError:
            continue
        except KeyError:
            continue
    financial_quarterly.append(balance)

In [29]:
financial_quarterly

[{'incomeBeforeTax': 13759160000,
  'minorityInterest': 1374586000,
  'netIncome': 12670968000,
  'sellingGeneralAdministrative': 21961211000,
  'grossProfit': 37194600000,
  'ebit': 17291355000,
  'endDate': 1640908800,
  'operatingIncome': 17291355000,
  'otherOperatingExpenses': -2057966000,
  'interestExpense': -1431774000,
  'incomeTaxExpense': 636578000,
  'totalRevenue': 72854344000,
  'totalOperatingExpenses': 55562989000,
  'costOfRevenue': 35659744000,
  'totalOtherIncomeExpenseNet': -3532195000,
  'netIncomeFromContinuingOps': 13122582000,
  'netIncomeApplicableToCommonShares': 12670968000},
 {'incomeBeforeTax': 13494441000,
  'minorityInterest': 1335496000,
  'netIncome': 11379394000,
  'sellingGeneralAdministrative': 17685269000,
  'grossProfit': 31312896000,
  'ebit': 16279757000,
  'endDate': 1609372800,
  'operatingIncome': 16279757000,
  'otherOperatingExpenses': -2652130000,
  'interestExpense': -1786600000,
  'incomeTaxExpense': 1762532000,
  'totalRevenue': 58378995

In [30]:
trimestre_index = []
trimestre_0 = pd.DataFrame.from_dict(financial_quarterly[0], orient="index").transpose()
trimestre_0 = trimestre_0.rename(index={0: pd.to_datetime(trimestre_0["endDate"], unit='s')[0]}) 
trimestre_index.append(pd.to_datetime(trimestre_0["endDate"], unit='s')[0])

trimestre_1 = pd.DataFrame.from_dict(financial_quarterly[1], orient="index").transpose()
trimestre_1 = trimestre_1.rename(index={0:  pd.to_datetime(trimestre_1["endDate"], unit='s')[0]}) 
trimestre_index.append(pd.to_datetime(trimestre_1["endDate"], unit='s')[0])

trimestre_2 = pd.DataFrame.from_dict(financial_quarterly[2], orient="index").transpose()
trimestre_2 = trimestre_2.rename(index={0: pd.to_datetime(trimestre_2["endDate"], unit='s')[0]}) 
trimestre_index.append(pd.to_datetime(trimestre_2["endDate"], unit='s')[0])

trimestre_3 = pd.DataFrame.from_dict(financial_quarterly[3], orient="index").transpose()
trimestre_3 = trimestre_3.rename(index={0:  pd.to_datetime(trimestre_3["endDate"], unit='s')[0]}) 
trimestre_index.append(pd.to_datetime(trimestre_3["endDate"], unit='s')[0])


In [31]:
income_statement = pd.concat([trimestre_0, trimestre_1, trimestre_2, trimestre_3])

In [32]:
income_statement

,incomeBeforeTax,minorityInterest,netIncome,sellingGeneralAdministrative,grossProfit,ebit,endDate,operatingIncome,otherOperatingExpenses,interestExpense,incomeTaxExpense,totalRevenue,totalOperatingExpenses,costOfRevenue,totalOtherIncomeExpenseNet,netIncomeFromContinuingOps,netIncomeApplicableToCommonShares
2021-12-31,13759160000,1374586000,12670968000,21961211000,37194600000,17291355000,1640908800,17291355000,-2057966000,-1431774000,636578000,72854344000,55562989000,35659744000,-3532195000,13122582000,12670968000
2020-12-31,13494441000,1335496000,11379394000,17685269000,31312896000,16279757000,1609372800,16279757000,-2652130000,-1786600000,1762532000,58378995000,42099238000,27066099000,-2785316000,11731909000,11379394000
2019-12-31,12943005000,1277980000,11779965000,15432767000,30326961000,16336142000,1577750400,16336142000,-1441948000,-1555873000,754673000,52005120000,35668978000,21678159000,-3393137000,12188332000,11779965000
2018-12-31,13121603000,1206801000,10994961000,14794989000,30981913000,17164064000,1546214400,17164064000,-977140000,-1495677000,1773893000,50231336000,33067272000,19249423000,-4042461000,11347710000,10994961000


In [33]:
# 6 Balance sheet

In [34]:
dados_json['context']['dispatcher']['stores']['QuoteSummaryStore'].keys()

dict_keys(['financialsTemplate', 'cashflowStatementHistory', 'balanceSheetHistoryQuarterly', 'earnings', 'price', 'incomeStatementHistoryQuarterly', 'incomeStatementHistory', 'balanceSheetHistory', 'cashflowStatementHistoryQuarterly', 'quoteType', 'summaryDetail', 'symbol', 'pageViews'])

In [35]:
balance_sheet = dados_json['context']['dispatcher']['stores']['QuoteSummaryStore']['balanceSheetHistory']['balanceSheetStatements']

In [36]:
balance_sheet

[{'intangibleAssets': {'raw': 8689015000,
   'fmt': '8.69B',
   'longFmt': '8,689,015,000'},
  'capitalSurplus': {'raw': 53662811000,
   'fmt': '53.66B',
   'longFmt': '53,662,811,000'},
  'totalLiab': {'raw': 54584866000,
   'fmt': '54.58B',
   'longFmt': '54,584,866,000'},
  'totalStockholderEquity': {'raw': 82643031000,
   'fmt': '82.64B',
   'longFmt': '82,643,031,000'},
  'minorityInterest': {'raw': 1374586000,
   'fmt': '1.37B',
   'longFmt': '1,374,586,000'},
  'otherCurrentLiab': {'raw': 5885492000,
   'fmt': '5.89B',
   'longFmt': '5,885,492,000'},
  'totalAssets': {'raw': 138602483000,
   'fmt': '138.6B',
   'longFmt': '138,602,483,000'},
  'endDate': {'raw': 1640908800, 'fmt': '2021-12-31'},
  'commonStock': {'raw': 58042464000,
   'fmt': '58.04B',
   'longFmt': '58,042,464,000'},
  'otherCurrentAssets': {'raw': 1680183000,
   'fmt': '1.68B',
   'longFmt': '1,680,183,000'},
  'retainedEarnings': {'raw': 31191640000,
   'fmt': '31.19B',
   'longFmt': '31,191,640,000'},
  'oth

In [37]:
balace_sheet_quarterly = []

In [38]:
for s in balance_sheet:
    balance = {}
    for key, val in s.items():
        try:
            balance[key] = val["raw"]
        except TypeError:
            continue
        except KeyError:
            continue
    balace_sheet_quarterly.append(balance)

In [39]:
balace_sheet_quarterly

[{'intangibleAssets': 8689015000,
  'capitalSurplus': 53662811000,
  'totalLiab': 54584866000,
  'totalStockholderEquity': 82643031000,
  'minorityInterest': 1374586000,
  'otherCurrentLiab': 5885492000,
  'totalAssets': 138602483000,
  'endDate': 1640908800,
  'commonStock': 58042464000,
  'otherCurrentAssets': 1680183000,
  'retainedEarnings': 31191640000,
  'otherLiab': 13465054000,
  'goodWill': 42411260000,
  'treasuryStock': -60253884000,
  'otherAssets': 19152749000,
  'cash': 16627698000,
  'totalCurrentLiabilities': 38866406000,
  'deferredLongTermAssetCharges': 4727749000,
  'shortLongTermDebt': 194080000,
  'otherStockholderEquity': -59216173000,
  'propertyPlantEquipment': 29224261000,
  'totalCurrentAssets': 38627141000,
  'longTermInvestments': 498057000,
  'netTangibleAssets': 31542756000,
  'netReceivables': 7404307000,
  'longTermDebt': 267137000,
  'inventory': 11000346000,
  'accountsPayable': 25077911000},
 {'intangibleAssets': 7580556000,
  'capitalSurplus': 536628

In [40]:
bs_trimestre_0 = pd.DataFrame.from_dict(balace_sheet_quarterly[0], orient="index").transpose()
bs_trimestre_0 = bs_trimestre_0.rename(index={0: pd.to_datetime(bs_trimestre_0["endDate"], unit='s')[0]}) 
bs_trimestre_1 = pd.DataFrame.from_dict(balace_sheet_quarterly[1], orient="index").transpose()
bs_trimestre_1 = bs_trimestre_1.rename(index={0: pd.to_datetime(bs_trimestre_1["endDate"], unit='s')[0]}) 
bs_trimestre_2 = pd.DataFrame.from_dict(balace_sheet_quarterly[2], orient="index").transpose()
bs_trimestre_2 = bs_trimestre_2.rename(index={0: pd.to_datetime(bs_trimestre_2["endDate"], unit='s')[0]}) 
bs_trimestre_3 = pd.DataFrame.from_dict(balace_sheet_quarterly[3], orient="index").transpose()
bs_trimestre_3 = bs_trimestre_3.rename(index={0: pd.to_datetime(bs_trimestre_3["endDate"], unit='s')[0]}) 

In [41]:
bs_trimestre = pd.concat([bs_trimestre_0, bs_trimestre_1, bs_trimestre_2, bs_trimestre_3])

In [42]:
bs_trimestre

,intangibleAssets,capitalSurplus,totalLiab,totalStockholderEquity,minorityInterest,otherCurrentLiab,totalAssets,endDate,commonStock,otherCurrentAssets,retainedEarnings,otherLiab,goodWill,treasuryStock,otherAssets,cash,totalCurrentLiabilities,deferredLongTermAssetCharges,shortLongTermDebt,otherStockholderEquity,propertyPlantEquipment,totalCurrentAssets,longTermInvestments,netTangibleAssets,netReceivables,longTermDebt,inventory,accountsPayable,shortTermInvestments
2021-12-31,8689015000,53662811000,54584866000,82643031000,1374586000,5885492000,138602483000,1640908800,58042464000,1680183000,31191640000,13465054000,42411260000,-60253884000,19152749000,16627698000,38866406000,4727749000,194080000,-59216173000,29224261000,38627141000,498057000,31542756000,7404307000,267137000,11000346000,25077911000,NaN
2020-12-31,7580556000,53662811000,50045455000,73815628000,1335496000,5924916000,125196579000,1609372800,57899073000,1356066000,25920061000,14514036000,40023457000,-63666317000,16930263000,17090335000,33477964000,4560808000,2332767000,-62724680000,24768355000,35342614000,551334000,26211615000,7590280000,338393000,7605905000,19339223000,NaN
2019-12-31,6306364000,53662811000,39186939000,61278025000,1277980000,4346739000,101742944000,1577750400,57866759000,1157704000,20874268000,11766251000,35009909000,-71125813000,9762242000,11900666000,25011033000,2950109000,217145000,-70170678000,22576299000,27621137000,466993000,19961752000,8569652000,697754000,5978557000,15069575000,NaN
2018-12-31,5840598000,53662811000,38259609000,56248007000,1206801000,4584454000,95714417000,1546214400,57710202000,680903000,15341367000,10888206000,34276176000,-70466373000,8225554000,11463498000,25208961000,2064742000,1530587000,-69583639000,21638008000,25329605000,404476000,16131233000,7028798000,850715000,5401793000,14050045000,13400000.0


In [43]:
ic_transpose = income_statement.transpose()
ic_transpose

,2021-12-31,2020-12-31,2019-12-31,2018-12-31
incomeBeforeTax,13759160000,13494441000,12943005000,13121603000
minorityInterest,1374586000,1335496000,1277980000,1206801000
netIncome,12670968000,11379394000,11779965000,10994961000
sellingGeneralAdministrative,21961211000,17685269000,15432767000,14794989000
grossProfit,37194600000,31312896000,30326961000,30981913000
ebit,17291355000,16279757000,16336142000,17164064000
endDate,1640908800,1609372800,1577750400,1546214400
operatingIncome,17291355000,16279757000,16336142000,17164064000
otherOperatingExpenses,-2057966000,-2652130000,-1441948000,-977140000
interestExpense,-1431774000,-1786600000,-1555873000,-1495677000


In [44]:
bs_transpose = bs_trimestre.transpose()
bs_transpose

,2021-12-31,2020-12-31,2019-12-31,2018-12-31
intangibleAssets,8.689015e+09,7.580556e+09,6.306364e+09,5.840598e+09
capitalSurplus,5.366281e+10,5.366281e+10,5.366281e+10,5.366281e+10
totalLiab,5.458487e+10,5.004546e+10,3.918694e+10,3.825961e+10
totalStockholderEquity,8.264303e+10,7.381563e+10,6.127802e+10,5.624801e+10
minorityInterest,1.374586e+09,1.335496e+09,1.277980e+09,1.206801e+09
otherCurrentLiab,5.885492e+09,5.924916e+09,4.346739e+09,4.584454e+09
totalAssets,1.386025e+11,1.251966e+11,1.017429e+11,9.571442e+10
endDate,1.640909e+09,1.609373e+09,1.577750e+09,1.546214e+09
commonStock,5.804246e+10,5.789907e+10,5.786676e+10,5.771020e+10
otherCurrentAssets,1.680183e+09,1.356066e+09,1.157704e+09,6.809030e+08


In [45]:
final = pd.concat([ic_transpose, bs_transpose])
final = final.transpose()

In [46]:
# 6 Balance sheet

In [47]:
import pandas_datareader as wp
from datetime import datetime, timedelta


In [48]:
trimestre_index[0] - timedelta(days=15)

Timestamp('2021-12-16 00:00:00')

In [49]:
stock_0 = wp.DataReader('ABEV3.SA', data_source="yahoo", start="2021-12-15",end="2021-12-15")
stock_0

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2021-12-15,16.26,15.94,16.26,16.08,21743100,15.476024


In [50]:
stock_0 = wp.DataReader(stock, data_source="yahoo", start=trimestre_index[0] - timedelta(days=15),end=trimestre_index[0] - timedelta(days=10))
stock_0 = stock_0.drop(stock_0.columns[[0,1,2,3,4]], axis = 1)

stock_1 = wp.DataReader(stock, data_source="yahoo", start=trimestre_index[1] - timedelta(days=15),end=trimestre_index[1] - timedelta(days=15))
stock_1 = stock_1.drop(stock_1.columns[[0,1,2,3,4]], axis = 1)

stock_2 = wp.DataReader(stock, data_source="yahoo", start=trimestre_index[2] - timedelta(days=15),end=trimestre_index[2] - timedelta(days=15))
stock_2 = stock_2.drop(stock_2.columns[[0,1,2,3,4]], axis = 1)

stock_3 = wp.DataReader(stock, data_source="yahoo", start=trimestre_index[3] - timedelta(days=12),end=trimestre_index[3] - timedelta(days=12))
stock_3 = stock_3.drop(stock_3.columns[[0,1,2,3,4]], axis = 1)
stock_0

,Adj Close
Date,
2021-12-16,15.495273
2021-12-17,15.466399
2021-12-20,15.420000
2021-12-21,15.400000


In [51]:
stock_0.iloc[0,0]

15.495272636413574

In [52]:
# Preco/Lucro

In [53]:
final

,incomeBeforeTax,minorityInterest,netIncome,sellingGeneralAdministrative,grossProfit,ebit,endDate,operatingIncome,otherOperatingExpenses,interestExpense,incomeTaxExpense,totalRevenue,totalOperatingExpenses,costOfRevenue,totalOtherIncomeExpenseNet,netIncomeFromContinuingOps,netIncomeApplicableToCommonShares,intangibleAssets,capitalSurplus,totalLiab,totalStockholderEquity,minorityInterest,otherCurrentLiab,totalAssets,endDate,commonStock,otherCurrentAssets,retainedEarnings,otherLiab,goodWill,treasuryStock,otherAssets,cash,totalCurrentLiabilities,deferredLongTermAssetCharges,shortLongTermDebt,otherStockholderEquity,propertyPlantEquipment,totalCurrentAssets,longTermInvestments,netTangibleAssets,netReceivables,longTermDebt,inventory,accountsPayable,shortTermInvestments
2021-12-31,1.375916e+10,1.374586e+09,1.267097e+10,2.196121e+10,3.719460e+10,1.729136e+10,1.640909e+09,1.729136e+10,-2.057966e+09,-1.431774e+09,6.365780e+08,7.285434e+10,5.556299e+10,3.565974e+10,-3.532195e+09,1.312258e+10,1.267097e+10,8.689015e+09,5.366281e+10,5.458487e+10,8.264303e+10,1.374586e+09,5.885492e+09,1.386025e+11,1.640909e+09,5.804246e+10,1.680183e+09,3.119164e+10,1.346505e+10,4.241126e+10,-6.025388e+10,1.915275e+10,1.662770e+10,3.886641e+10,4.727749e+09,1.940800e+08,-5.921617e+10,2.922426e+10,3.862714e+10,498057000.0,3.154276e+10,7.404307e+09,267137000.0,1.100035e+10,2.507791e+10,NaN
2020-12-31,1.349444e+10,1.335496e+09,1.137939e+10,1.768527e+10,3.131290e+10,1.627976e+10,1.609373e+09,1.627976e+10,-2.652130e+09,-1.786600e+09,1.762532e+09,5.837900e+10,4.209924e+10,2.706610e+10,-2.785316e+09,1.173191e+10,1.137939e+10,7.580556e+09,5.366281e+10,5.004546e+10,7.381563e+10,1.335496e+09,5.924916e+09,1.251966e+11,1.609373e+09,5.789907e+10,1.356066e+09,2.592006e+10,1.451404e+10,4.002346e+10,-6.366632e+10,1.693026e+10,1.709034e+10,3.347796e+10,4.560808e+09,2.332767e+09,-6.272468e+10,2.476836e+10,3.534261e+10,551334000.0,2.621162e+10,7.590280e+09,338393000.0,7.605905e+09,1.933922e+10,NaN
2019-12-31,1.294300e+10,1.277980e+09,1.177996e+10,1.543277e+10,3.032696e+10,1.633614e+10,1.577750e+09,1.633614e+10,-1.441948e+09,-1.555873e+09,7.546730e+08,5.200512e+10,3.566898e+10,2.167816e+10,-3.393137e+09,1.218833e+10,1.177996e+10,6.306364e+09,5.366281e+10,3.918694e+10,6.127802e+10,1.277980e+09,4.346739e+09,1.017429e+11,1.577750e+09,5.786676e+10,1.157704e+09,2.087427e+10,1.176625e+10,3.500991e+10,-7.112581e+10,9.762242e+09,1.190067e+10,2.501103e+10,2.950109e+09,2.171450e+08,-7.017068e+10,2.257630e+10,2.762114e+10,466993000.0,1.996175e+10,8.569652e+09,697754000.0,5.978557e+09,1.506958e+10,NaN
2018-12-31,1.312160e+10,1.206801e+09,1.099496e+10,1.479499e+10,3.098191e+10,1.716406e+10,1.546214e+09,1.716406e+10,-9.771400e+08,-1.495677e+09,1.773893e+09,5.023134e+10,3.306727e+10,1.924942e+10,-4.042461e+09,1.134771e+10,1.099496e+10,5.840598e+09,5.366281e+10,3.825961e+10,5.624801e+10,1.206801e+09,4.584454e+09,9.571442e+10,1.546214e+09,5.771020e+10,6.809030e+08,1.534137e+10,1.088821e+10,3.427618e+10,-7.046637e+10,8.225554e+09,1.146350e+10,2.520896e+10,2.064742e+09,1.530587e+09,-6.958364e+10,2.163801e+10,2.532960e+10,404476000.0,1.613123e+10,7.028798e+09,850715000.0,5.401793e+09,1.405004e+10,13400000.0


In [54]:
#amount stock

amount_shares_0 = dados_json['context']['dispatcher']['stores']['QuoteTimeSeriesStore']["timeSeries"]['annualBasicAverageShares'][0]["reportedValue"]["raw"]
amount_shares_1 = dados_json['context']['dispatcher']['stores']['QuoteTimeSeriesStore']["timeSeries"]['annualBasicAverageShares'][1]["reportedValue"]["raw"]
amount_shares_2 = dados_json['context']['dispatcher']['stores']['QuoteTimeSeriesStore']["timeSeries"]['annualBasicAverageShares'][2]["reportedValue"]["raw"]
amount_shares_3 = dados_json['context']['dispatcher']['stores']['QuoteTimeSeriesStore']["timeSeries"]['annualBasicAverageShares'][3]["reportedValue"]["raw"]
final["amount_shares"] = [amount_shares_0, amount_shares_1, amount_shares_2, amount_shares_3]

price_stock_0 = stock_0.iloc[0,0]
price_stock_1 = stock_1.iloc[0,0]
price_stock_2 = stock_2.iloc[0,0]
price_stock_3 = stock_3.iloc[0,0]
final["price_stock"] = [price_stock_0,price_stock_1,price_stock_2,price_stock_3]

In [55]:
lucro_liquido = ic_transpose.iloc[-1,:]
#netIncomeApplicableToCommonShares
lucro_liquido

2021-12-31    12670968000
2020-12-31    11379394000
2019-12-31    11779965000
2018-12-31    10994961000
Name: netIncomeApplicableToCommonShares, dtype: int64

In [56]:
final["netIncomeApplicableToCommonShares"]

2021-12-31    1.267097e+10
2020-12-31    1.137939e+10
2019-12-31    1.177996e+10
2018-12-31    1.099496e+10
Name: netIncomeApplicableToCommonShares, dtype: float64

In [57]:
#LPA - Lucro por ação
final["LPA"] = final["netIncomeApplicableToCommonShares"]/final["amount_shares"]
final["LPA"]

2021-12-31    0.806139
2020-12-31    0.723535
2019-12-31    0.748738
2018-12-31    0.698674
Name: LPA, dtype: float64

In [58]:
#Preço/lucro
final["preco_lucro"] = final["price_stock"]/final["LPA"]
final["preco_lucro"]

2021-12-31    19.221597
2020-12-31    20.404295
2019-12-31    23.172029
2018-12-31    20.042946
Name: preco_lucro, dtype: float64

In [59]:
#P/VP valor da empresa
final["P/VP"] = final["price_stock"]*final["amount_shares"]
final["P/VP"] 

2021-12-31    2.435562e+11
2020-12-31    2.321885e+11
2019-12-31    2.729657e+11
2018-12-31    2.203714e+11
Name: P/VP, dtype: float64

In [60]:
#PL
final["PL"] = final["totalCurrentLiabilities"]*final["amount_shares"]
final["PL"]

2021-12-31    6.109061e+20
2020-12-31    5.265247e+20
2019-12-31    3.935011e+20
2018-12-31    3.967109e+20
Name: PL, dtype: float64

In [61]:
#P/Ativos preco por valor ativo totalAssets
final["P/ATIVO"] = final["price_stock"]/(final["totalAssets"]/final["amount_shares"])
final["P/ATIVO"]

2021-12-31    1.757229
2020-12-31    1.854591
2019-12-31    2.682896
2018-12-31    2.302385
Name: P/ATIVO, dtype: float64

In [62]:
#VPA é o patrimônio líquido da empresa dividido pelo número de ações.
final["VPA"] = final["totalStockholderEquity"]/final["amount_shares"]
final["VPA"]

2021-12-31    5.257826
2020-12-31    4.693411
2019-12-31    3.894847
2018-12-31    3.574275
Name: VPA, dtype: float64

In [63]:
#VM = Número de ações emitidas X Cotação
final["VM"] = final["amount_shares"]*final["price_stock"]
final["VM"]

2021-12-31    2.435562e+11
2020-12-31    2.321885e+11
2019-12-31    2.729657e+11
2018-12-31    2.203714e+11
Name: VM, dtype: float64

In [64]:
#EV Valor de mercado (EV) 
final["EV"] = final["VM"]+final["shortLongTermDebt"]-final["cash"]
final["EV"]

2021-12-31    2.271226e+11
2020-12-31    2.174309e+11
2019-12-31    2.612822e+11
2018-12-31    2.104385e+11
Name: EV, dtype: float64

In [65]:
#EV/EBIT Valor de mercado (EV) / Lucro Antes de Juros e Impostos (EBIT)
final["EV/EBIT"] = final["EV"]/final["ebit"]
final["EV/EBIT"]

2021-12-31    13.135039
2020-12-31    13.355908
2019-12-31    15.994117
2018-12-31    12.260412
Name: EV/EBIT, dtype: float64

In [66]:
#NOPAT (Net Operating Profit After Taxes)
final["NOPAT"] = final["ebit"]-final["incomeTaxExpense"]
final["NOPAT"]

2021-12-31    1.665478e+10
2020-12-31    1.451722e+10
2019-12-31    1.558147e+10
2018-12-31    1.539017e+10
Name: NOPAT, dtype: float64

In [67]:
#ROIC = NOPLAT/ Capital investido
final["ROIC"] = final["NOPAT"]/final["longTermInvestments"]
final["ROIC"]

2021-12-31    33.439500
2020-12-31    26.331090
2019-12-31    33.365530
2018-12-31    38.049652
Name: ROIC, dtype: float64

In [68]:
final["formula-gharant"] = (final["VPA"]*final["LPA"]*22.5)**0.5

In [69]:
final["margem-seguranca"] = (final["formula-gharant"]/final["price_stock"]-1)*100

In [70]:
final

,incomeBeforeTax,minorityInterest,netIncome,sellingGeneralAdministrative,grossProfit,ebit,endDate,operatingIncome,otherOperatingExpenses,interestExpense,incomeTaxExpense,totalRevenue,totalOperatingExpenses,costOfRevenue,totalOtherIncomeExpenseNet,netIncomeFromContinuingOps,netIncomeApplicableToCommonShares,intangibleAssets,capitalSurplus,totalLiab,totalStockholderEquity,minorityInterest,otherCurrentLiab,totalAssets,endDate,commonStock,otherCurrentAssets,retainedEarnings,otherLiab,goodWill,treasuryStock,otherAssets,cash,totalCurrentLiabilities,deferredLongTermAssetCharges,shortLongTermDebt,otherStockholderEquity,propertyPlantEquipment,totalCurrentAssets,longTermInvestments,netTangibleAssets,netReceivables,longTermDebt,inventory,accountsPayable,shortTermInvestments,amount_shares,price_stock,LPA,preco_lucro,P/VP,PL,P/ATIVO,VPA,VM,EV,EV/EBIT,NOPAT,ROIC,formula-gharant,margem-seguranca
2021-12-31,1.375916e+10,1.374586e+09,1.267097e+10,2.196121e+10,3.719460e+10,1.729136e+10,1.640909e+09,1.729136e+10,-2.057966e+09,-1.431774e+09,6.365780e+08,7.285434e+10,5.556299e+10,3.565974e+10,-3.532195e+09,1.312258e+10,1.267097e+10,8.689015e+09,5.366281e+10,5.458487e+10,8.264303e+10,1.374586e+09,5.885492e+09,1.386025e+11,1.640909e+09,5.804246e+10,1.680183e+09,3.119164e+10,1.346505e+10,4.241126e+10,-6.025388e+10,1.915275e+10,1.662770e+10,3.886641e+10,4.727749e+09,1.940800e+08,-5.921617e+10,2.922426e+10,3.862714e+10,498057000.0,3.154276e+10,7.404307e+09,267137000.0,1.100035e+10,2.507791e+10,NaN,15718100000,15.495273,0.806139,19.221597,2.435562e+11,6.109061e+20,1.757229,5.257826,2.435562e+11,2.271226e+11,13.135039,1.665478e+10,33.439500,9.765607,-36.976865
2020-12-31,1.349444e+10,1.335496e+09,1.137939e+10,1.768527e+10,3.131290e+10,1.627976e+10,1.609373e+09,1.627976e+10,-2.652130e+09,-1.786600e+09,1.762532e+09,5.837900e+10,4.209924e+10,2.706610e+10,-2.785316e+09,1.173191e+10,1.137939e+10,7.580556e+09,5.366281e+10,5.004546e+10,7.381563e+10,1.335496e+09,5.924916e+09,1.251966e+11,1.609373e+09,5.789907e+10,1.356066e+09,2.592006e+10,1.451404e+10,4.002346e+10,-6.366632e+10,1.693026e+10,1.709034e+10,3.347796e+10,4.560808e+09,2.332767e+09,-6.272468e+10,2.476836e+10,3.534261e+10,551334000.0,2.621162e+10,7.590280e+09,338393000.0,7.605905e+09,1.933922e+10,NaN,15727500000,14.763218,0.723535,20.404295,2.321885e+11,5.265247e+20,1.854591,4.693411,2.321885e+11,2.174309e+11,13.355908,1.451722e+10,26.331090,8.741084,-40.791472
2019-12-31,1.294300e+10,1.277980e+09,1.177996e+10,1.543277e+10,3.032696e+10,1.633614e+10,1.577750e+09,1.633614e+10,-1.441948e+09,-1.555873e+09,7.546730e+08,5.200512e+10,3.566898e+10,2.167816e+10,-3.393137e+09,1.218833e+10,1.177996e+10,6.306364e+09,5.366281e+10,3.918694e+10,6.127802e+10,1.277980e+09,4.346739e+09,1.017429e+11,1.577750e+09,5.786676e+10,1.157704e+09,2.087427e+10,1.176625e+10,3.500991e+10,-7.112581e+10,9.762242e+09,1.190067e+10,2.501103e+10,2.950109e+09,2.171450e+08,-7.017068e+10,2.257630e+10,2.762114e+10,466993000.0,1.996175e+10,8.569652e+09,697754000.0,5.978557e+09,1.506958e+10,NaN,15733100000,17.349771,0.748738,23.172029,2.729657e+11,3.935011e+20,2.682896,3.894847,2.729657e+11,2.612822e+11,15.994117,1.558147e+10,33.365530,8.100304,-53.311752
2018-12-31,1.312160e+10,1.206801e+09,1.099496e+10,1.479499e+10,3.098191e+10,1.716406e+10,1.546214e+09,1.716406e+10,-9.771400e+08,-1.495677e+09,1.773893e+09,5.023134e+10,3.306727e+10,1.924942e+10,-4.042461e+09,1.134771e+10,1.099496e+10,5.840598e+09,5.366281e+10,3.825961e+10,5.624801e+10,1.206801e+09,4.584454e+09,9.571442e+10,1.546214e+09,5.771020e+10,6.809030e+08,1.534137e+10,1.088821e+10,3.427618e+10,-7.046637e+10,8.225554e+09,1.146350e+10,2.520896e+10,2.064742e+09,1.530587e+09,-6.958364e+10,2.163801e+10,2.532960e+10,404476000.0,1.613123e+10,7.028798e+09,850715000.0,5.401793e+09,1.405004e+10,13400000.0,15736900000,14.003483,0.698674,20.042946,2.203714e+11,3.967109e+20,2.302385,3.574275,2.203714e+11,2.104385e+11,12.260412,1.539017e+10,38.049652,7.495878,-46.471333


In [71]:
fileName = "{}.csv"
fileName = fileName.format(stock)
final.to_csv(fileName)